In [44]:
%matplotlib inline 
import matplotlib.pyplot as plt
import numpy as np 
import glob as glob
import random
from statistics import mode
from numpy import trapz
from astropy.time import Time
c = 299792 #speed of light in km/s

In [8]:
def gsmooth(x_array, y_array, var_y, vexp = .001, nsig = 5.0):
    #returns smoothed flux array
    # if no variance array create a constant one
    if len(var_y) == 0:
        var_y = np.zeros(len(y_array))
        
    for i in range(len(var_y)):
        if var_y[i] == 0:
            var_y[i] = 1E-20
            # var_y[i] = 1E-31
    
    # Output y-array
    new_y = np.zeros(len(x_array), float)
    
    # Loop over y-array elements
    for i in range(len(x_array)):
        
        # Construct a Gaussian of sigma = vexp*x_array[i]
        gaussian = np.zeros(len(x_array), float)
        sigma = vexp*x_array[i]
        
        # Restrict range to +/- nsig sigma
        sigrange = np.nonzero(abs(x_array-x_array[i]) <= nsig*sigma)
        gaussian[sigrange] = (1/(sigma*np.sqrt(2*np.pi)))*np.exp(-0.5*((x_array[sigrange]-x_array[i])/sigma)**2)
        
        # Multiply Gaussian by 1 / variance
        W_lambda = gaussian / var_y
        
        # Perform a weighted sum to give smoothed y value at x_array[i]
        W0 = np.sum(W_lambda)
        W1 = np.sum(W_lambda*y_array)
        new_y[i] = W1/W0

    # Return smoothed y-array
    return new_y

In [9]:
def deredshift(wavelength, z):
    dered_wave = wavelength / (1+z)
    return dered_wave

In [10]:
def find_min_wave(dered_wave, sm_flux, w1 = 5900., w2 = 6500.):
    elem_range = np.where((dered_wave > w1) & (dered_wave < w2)) #set domain as within minwave and maxwave
    list(elem_range)
    elem_wavelist = dered_wave[elem_range] 
    elem_fluxlist = sm_flux[elem_range] #find corresponding flux vales for wavelengths within domain
    elem_flux = min(elem_fluxlist) #find minimum value within these flux vales to locate "dip"
    elem_wavelength = dered_wave[np.where(sm_flux == elem_flux)][0] #find the corresponding wavelength
    return(elem_wavelength)

In [11]:
def find_equivalent_width(dered_wave, sm_flux, w1 = 5700., w2 = 6100., w3 = 6300., w4 = 6550.):
    max_1 = np.where((dered_wave > w1) & (dered_wave < w2))
    list(max_1)
    elem_wavelist_1 = dered_wave[max_1]
    elem_fluxlist_1 = sm_flux[max_1]
    elem_flux_1 = max(elem_fluxlist_1)
    max_1_wave = dered_wave[np.where(sm_flux == elem_flux_1)][0]
    
    max_2 = np.where((dered_wave > w3) & (dered_wave < w4))
    list(max_2)
    elem_wavelist_2 = dered_wave[max_2]
    elem_fluxlist_2 = sm_flux[max_2]
    elem_flux_2 = max(elem_fluxlist_2)
    max_2_wave = dered_wave[np.where(sm_flux == elem_flux_2)][0]
    
    eqw_domain = np.where((dered_wave >= max_1_wave) & (dered_wave <= max_2_wave))
    list(eqw_domain)
    elem_fluxlist_eqw_domain = sm_flux[eqw_domain]
    elem_wavelist_eqw_domain = dered_wave[eqw_domain]
    
    m = ((elem_flux_2 - elem_flux_1)/(max_2_wave - max_1_wave))
    b = elem_flux_2 - m*(max_2_wave)
    y = m*(elem_wavelist_eqw_domain) + b
    
    area_small = trapz(elem_fluxlist_eqw_domain, dx=1)
    area_big = trapz(y, dx =1)
    
    equivalent_width = area_big - area_small
    
    return (equivalent_width)

In [12]:
eq_width = find_equivalent_width(dered_wave, sm_flux)
print(eq_width)

NameError: name 'dered_wave' is not defined

In [18]:
def find_velocity(min_wave, rest_wave = 6355.):
    wavelength_prop = min_wave / rest_wave
    ejecta_velocity = c * (wavelength_prop ** 2 - 1) / (wavelength_prop ** 2 + 1)
    return ejecta_velocity

In [ ]:
def get_SN_name(file):
    SN_name = file.split('-')[0]
    return SN_name

In [ ]:
def make_z_dict(file):
    #reads in CfA4_info.txt
    sn_datatext = open(file, "r")
    sn_datainfo = sn_datatext.readlines()
    z_dict = {}
    start_line = 60
    for line in sn_datainfo[start_line:]:
        line_elems = line.split()
        temp = line_elems[0]
        if temp[0] == "2":
            name = "sn" + temp
        else:
            name = temp
        z = line_elems[10]
        z_dict[name] = float(z)
    return z_dict

In [ ]:
def vel_error_dist(wavelength, flux, flux_error, n = 1000):
    velocity_distribution = []
    minwave = 6000
    maxwave = 7000
    wavelength_rest = 6355
    for i in range(n):
        vexp= np.random.normal(.0015,.00025) 
        smooth_flux = gsmooth(wavelength, flux, flux_error,vexp)
        SiII_obs = find_wave_obs(wavelength, smooth_flux, minwave, maxwave)
        v = find_velocity(SiII_obs, wavelength_rest)
        #print (v, vexp, SiII_obs) this line can be uncommented to analyze increments of the loop individually
        velocity_distribution.append(v)
    vmode = mode(velocity_distribution)
    vmean = np.mean(velocity_distribution)
    vstd = np.std(velocity_distribution)
    lower95bound = sorted(velocity_distribution)[24]
    upper95bound = sorted(velocity_distribution)[974]
    plt.hist(velocity_distribution)
    return "Ejecta velocity is", vmode ,"km/s", "mean velocity is", vmean, "km/s", "standard deviation is", vstd, "km/s.", "The 95th percentile confidence interval is between", lower95, "and", upper95

In [ ]:
def dflux(wave):
    unsmflux = flux[np.where(dered_wavelength == wave)]
    smflux = sm_flux[np.where(dered_wavelength == wave)]
    dflux = smflux - unsmflux
    return dflux

In [ ]:
#Parse ned for associated galaxy
#use astroquery to find z value
def find_z(SN_name):
    query_url = 'https://ned.ipac.caltech.edu/cgi-bin/objsearch?objname=sn2007fs&extend=no&hconst=73&omegam=0.27&omegav=0.73&corr_z=1&out_csys=Equatorial&out_equinox=J2000.0&obj_sort=RA+or+Longitude&of=pre_text&zv_breaker=30000.0&list_limit=5&img_stamp=YES'
    sn_url = query_url.replace("sn2007fs", SN_name)
    return sn_url

In [13]:
def plotspectra(wavelength, flux, dered_wave, sm_flux):
    plt.figure(figsize=(20, 10))
    plt.axvline(x=6355, linewidth=.5)
    #resting wavelength of Si II
    plt.plot(wavelength, flux, linewidth=1, color='red') #redshifted
    plt.plot(dered_wave, sm_flux, linewidth=1, color='blue') #deredshifted
    plt.xlabel('Wavelength (Angstroms)')
    plt.ylabel('Relative Flux')

In [14]:
#TEST YOUR CODE ON ONE OBJECT (plot original spectrum, deredshifted spectrum, and the smoothed deredshifted spectrum, print velocity)
#YOUR CODE GOES HERE
my_Supernova = ("sn2007hj-20070903.28-fast.flm")

wavelength, flux, flux_error = np.loadtxt(my_Supernova, unpack = True)
z = 0.003
dered_wave = deredshift(wavelength, z)
sm_flux = gsmooth(dered_wave, flux, flux_error)

plotspectra(wavelength, flux, dered_wave, sm_flux)

plt.axhline(y = 9e-16)


FileNotFoundError: [Errno 2] No such file or directory: 'sn2007hj-20070903.28-fast.flm'

In [15]:
#YOUR CODE GOES HERE
z_dict = make_z_dict("CfA4_info.txt")
sndata = glob.glob("*.flm")
wavelength, flux, flux_error = np.loadtxt(sndata[0], unpack = True)
z = 0.017192 #obtain from NED
#c = 299792 #speed of light in km/s
print(sndata)
dered_wave = deredshift(wavelength, z)
sm_flux = gsmooth(dered_wave, flux, flux_error)
min_wave = find_min_wave(dered_wave, sm_flux)
print(min_wave)
v = find_velocity(min_wave)
print(v)

NameError: name 'make_z_dict' is not defined

In [16]:
make_z_dict("CfA4_info.txt")

NameError: name 'make_z_dict' is not defined

In [34]:
class SupernovaName:
    def __init__(self, SN_name):
        self.name = SN_name
        self.files = glob.glob(self.name)
        #self.z = code working externally from Hyperion

In [46]:
#ERROR for some reason glob is reading in 'filename' as a file object or a variable and not a filename :/
class spectrum:
    def __init__(self, filename):
        self.filename = filename
        self.data = glob.glob(self.filename)
        wavelength, flux, flux_error = np.loadtxt(self.data[0], unpack = True)
        
    def csndata(self):
        sndata = glob.glob(self.filename)
        wavelength, flux, flux_error = np.loadtxt(sndata[0], unpack = True)
        plt.figure(figsize=(10, 4))
        plt.plot(wavelength,flux, linewidth=.5, color='blue')
        plt.axvline(x=6355, linewidth=.5)
        #resting wavelength of Si II
        plt.xlabel('Wavelength (Angstroms)')
        plt.ylabel('Relative Flux')
        return wavelength, flux, flux_error
    
    def cderedshift(self):
        #z = get_z(name) must be implemented later
        sndata = glob.glob(self.filename)
        wavelength, flux, flux_error = np.loadtxt(sndata[0], unpack = True)
        dered_wave = deredshift(wavelength, z)
        return dered_wave
    
    def cfind_min_wave(self):
        #z = get_z(name) must be implemented later
        z = .017193 #test
        sndata = glob.glob(self.filename)
        wavelength, flux, flux_error = np.loadtxt(sndata[0], unpack = True)
        dered_wave = deredshift(wavelength, z)
        sm_flux = gsmooth(dered_wave, flux, flux_error) #automated smoothing has not been implemented yet
        min_wave = find_min_wave(dered_wave, sm_flux)
        return min_wave
    
    def cfind_velocity(self):
        z = .017193 #test
        sndata = glob.glob(self.filename)
        wavelength, flux, flux_error = np.loadtxt(sndata[0], unpack = True)
        dered_wave = deredshift(wavelength, z)
        sm_flux = gsmooth(dered_wave, flux, flux_error) #automated smoothing has not been implemented yet
        min_wave = find_min_wave(dered_wave, sm_flux)
        ejecta_velocity = find_velocity(min_wave)
        return ejecta_velocity
    
sn = spectrum('sn2007fsdata.flm')
print (sn.cfind_velocity())

-10837.172779


In [3]:
#FINAL SCRIPT STEPS
#1) Get array of all flm file names
#2) Make redshift dictionary
#3) loop through files
    #a) read in spectrum
    #b) get SN name from file name
    #c) get redshift from dictionary
    #d) deredshift
    #e) smooth
    #f) find wavelength where Si line is minimum
    #g) find and print velocity (and SN name)

In [4]:
sndata = glob.glob("*.flm")

z_dict = make_z_dict("CfA4_info.txt")
for i in range(len(sndata)):
    name = get_SN_name(sndata[i])
    wavelength, flux, flux_error = np.loadtxt(sndata[i], unpack = True)
    dered_wave = deredshift(wavelength, z_dict[name])
    sm_flux = gsmooth(dered_wave, flux, flux_error)
    min_wave = find_min_wave(dered_wave, sm_flux)
    v = find_velocity(min_wave)
    print (v)

NameError: name 'glob' is not defined

In [5]:
def find_t_obs(file): #gets observed date and time from file name and converts it to a modified julian date
    date = file.split('-')[1]
    datetime = date[0:4] + "-" + date[4:6] + "-" + date[6:]
    t = Time(datetime)
    t.format = 'mjd'    
    t_obs = t.value
    return t_obs
    
#test it out
find_t_obs("sn2007jg-20070920.46-fast.flm")

NameError: name 'Time' is not defined

In [6]:
def measure_phase(file, t_max, z_dict): #get t_max from Matt
    t_obs = find_t_obs(file)
    z = z_dict[get_SN_name(file)]
    phase = (t_obs - t_max)/(1+z)
    return phase

#random test with random t_max, I don't know what t-max is yet
measure_phase("sn2007jg-20070920.46-fast.flm", 53000, z_dict)

NameError: name 'z_dict' is not defined

In [33]:
def find_v_0 (phase, v_obs, sigma_vobs): #computes velocity at t_max
    v = v_obs
    t = phase
    v_0 = (v + (0.285*t))/(1 - (0.0322*t))
    sigma_v_0 = ((sigma_v_obs/(1-0.0322t))^2 + 0.22^2)^(1/2) #get sigma_vobs from the kyle's function that determines the error of a velocity measurement
    return v_0, sigma_v_0